In [ ]:
%load_ext autoreload
%autoreload 2

## DB Setup

We assume you already have a sqlite database ready.

In [ ]:
DATABASE = "testDB.db"
TABLES = []  # list of tables to load or [] to load all tables

In [ ]:
from db_connectors import SQLiteConnector
from prompt_formatters import RajkumarFormatter

# Get the connector and formatter
sqlite_connector = SQLiteConnector(
    database_path=DATABASE
)
sqlite_connector.connect()
if len(TABLES) <= 0:
    TABLES.extend(sqlite_connector.get_tables())

print(f"Loading tables: {TABLES}")

db_schema = [sqlite_connector.get_schema(table) for table in TABLES]
formatter = RajkumarFormatter(db_schema)

## Model Setup

In a separate screen or window, first install [Manifest](https://github.com/HazyResearch/manifest)
```bash
pip install manifest-ml\[all\]
```

Then run
```bash
python3 -m manifest.api.app \
    --model_type huggingface \
    --model_generation_type text-generation \
    --model_name_or_path NumbersStation/nsql-350M \
    --device 0
```

If successful, you will see an output like
```bash
* Running on http://127.0.0.1:5000
```

In [ ]:
from manifest import Manifest

manifest_client = Manifest(client_name="huggingface", client_connection="http://127.0.0.1:5000")

def get_sql(instruction: str, max_tokens: int = 300) -> str:
    prompt = formatter.format_prompt(instruction)
    res = manifest_client.run(prompt, max_tokens=max_tokens)
    return formatter.format_model_output(res)

In [ ]:
sql = get_sql("Number of rows in table?")
print(sql)

In [ ]:
print(sqlite_connector.run_sql_as_df(sql))